# Glauber on infinite random $k$-regular graph
Compute magnetization in the thermodynamic limit $N\to\infty$ for random $k$-regular graphs with $k=8$.

Compare the steady-state magnetization with its equilibrium value.

In [2]:
using MatrixProductBP, MatrixProductBP.Models
using Plots
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using JLD2
using LaTeXStrings;

In [58]:
T = 25         # final time
k = 8          # degree
m⁰ = 0.3       # magnetization at time zero

β = 1.0
J = 0.2
h = 0.0;

In [59]:
m_eq, = equilibrium_observables(RandomRegular(k), J; β, h)

(m = 0.8596433979493756, r = 0.7570683387764126, e = -0.6056546710211301)

In [60]:
wᵢ = fill(HomogeneousGlauberFactor(J, h, β), T+1)
ϕᵢ = [ t == 0 ? [(1+m⁰)/2, (1-m⁰)/2] : ones(2) for t in 0:T]
bp = mpbp_infinite_graph(k, wᵢ, 2, ϕᵢ);

In [ ]:
matrix_sizes = [2, 5, 10, 25]
maxiters = fill(50, length(matrix_sizes))
iters = zeros(Int, length(maxiters))
tol = 1e-4
spin(x, i) = 3-2x
m = Vector{Vector{Float64}}(undef, length(matrix_sizes))
for i in eachindex(maxiters)
    reset!(bp)
    for j in 1:i
        iters[i], _ = iterate!(bp; maxiter=maxiters[j], svd_trunc=TruncBond(matrix_sizes[j]), tol)
    end
    m[i] = only(means(spin, bp))
end

Running MPBP: iter 15 	 Time: 0:01:16
  Δ:                8.680811601036176e-5
  SVD Matrix size:  10

In [ ]:
using ColorSchemes

cg = cgrad(:blues, length(matrix_sizes), categorical=true) |> reverse
pl = hline([m_eq], c=:red, ls=:dash, lw=1, label="equilibrium")
for i in eachindex(matrix_sizes)
    plot!(pl, 0:T, m[i], xlabel="time", ylabel="magnetization", label="MPBP - $(matrix_sizes[i])",
        size=(500,300), xticks=0:2:T, ms=3, title="Glauber infinite $k-regular", titlefontsize=12,
        legend=:bottomright, msc=:auto, c=cg[i])
end
pl

### Compare with Monte Carlo on finite graphs

In [29]:
using Graphs, IndexedGraphs, Statistics

N = 5*10^3
g = random_regular_graph(N, k)
ising = Ising(IndexedGraph(g); J=fill(J, ne(g)), h=fill(h, N), β)
bp_mc = mpbp(Glauber(ising, T); ϕ = fill(ϕᵢ, N))
sms = SoftMarginSampler(bp_mc);

In [30]:
sample!(sms, 10^3)
spin(x) = 3-2x
m_mc = [vec(spin.(mean(X, dims=1))) for X in sms.X];

SoftMargin sampling 100%|████████████████████████████████| Time: 0:00:22


In [ ]:
plot!(pl, 0:T, mean(m_mc), c=:black, m=:diamond, ms=3, msc=:auto, label="", st=:scatter)

In [ ]:
# jldsave("plot_data/glauber_infinite8RRG_bonddims.jld2"; T, k, m_mc, m, m_eq);